<a href="https://colab.research.google.com/github/kyulee-jeon/Tutorial/blob/main/Week12_(2)ResNet_Transfer_Wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Loading**

In [ ]:
#Connection to GoogleDrive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
from glob import glob
import numpy as np
import pickle

In [ ]:
data_path = "/content/gdrive/MyDrive/Colab Notebooks/2022MedicalMachineLearning/data/COVID-19_Radiography_Dataset"
"""
#Define the file paths
all_covid_paths = [f for f in glob(os.path.join(data_path, 'COVID', 'images', 'COVID*.png'))]
#all_opac_paths = [f for f in glob(os.path.join(data_path, 'Lung_Opacity', 'images', 'Lung_Opacity*.png'))]
all_normal_paths = [f for f in glob(os.path.join(data_path, 'Normal', 'images', 'Normal*.png'))]
# all_data_paths = all_covid_paths + all_opac_paths + all_normal_paths
"""


"\n#Define the file paths\nall_covid_paths = [f for f in glob(os.path.join(data_path, 'COVID', 'images', 'COVID*.png'))]\n#all_opac_paths = [f for f in glob(os.path.join(data_path, 'Lung_Opacity', 'images', 'Lung_Opacity*.png'))]\nall_normal_paths = [f for f in glob(os.path.join(data_path, 'Normal', 'images', 'Normal*.png'))]\n# all_data_paths = all_covid_paths + all_opac_paths + all_normal_paths\n"

In [ ]:
#load the list
with open(os.path.join(data_path, "list_x_wk10"), "rb") as f:
  list_x = pickle.load(f)
with open(os.path.join(data_path, "list_y_wk10"), "rb") as f:
  list_y = pickle.load(f)

In [ ]:
#Gray scale to RGB
import cv2
list_x = list(map(lambda x: cv2.cvtColor(x,cv2.COLOR_GRAY2RGB), list_x))

In [ ]:
#convert to ndarray of the data
data_x = np.array(list_x)
data_y = np.array(list_y)

In [ ]:
#save the array
"""
with open(os.path.join(data_path, "data_x_wk11"), "wb") as f:
  pickle.dump(data_x, f)
with open(os.path.join(data_path, "list_y_wk11"), "wb") as f:
  pickle.dump(data_y, f)
"""

'\nwith open(os.path.join(data_path, "data_x_wk11"), "wb") as f:\n  pickle.dump(data_x, f)\nwith open(os.path.join(data_path, "list_y_wk11"), "wb") as f:\n  pickle.dump(data_y, f)\n'

In [ ]:
print("The length of list_x is {}".format(len(list_x)))
print("The length of list_y is {}".format(len(list_y)))
print("The shape of first element of list_x is {}".format(list_x[0].shape))

del list_x, list_y

The length of list_x is 8616
The length of list_y is 8616
The shape of first element of list_x is (299, 299, 3)


In [ ]:
#one hot encoding
data_y_ = data_y.reshape(-1,1)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe.fit(data_y_)
data_y = ohe.transform(data_y_)

Data Preprocessing

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
# split into train, validation, and test sets (stratified)
#how to split validation and test
#https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn

random_seed = 11
train_prop = 0.6
val_prop = 0.2
test_prop = 0.2

from sklearn.model_selection import train_test_split
x_train, x_val_test, y_train, y_val_test = train_test_split(data_x, data_y, 
                                                            test_size = val_prop+test_prop, 
                                                            random_state = random_seed, 
                                                            stratify = data_y)

x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test,
                                                test_size = test_prop/(val_prop+test_prop),
                                                random_state = random_seed,
                                                stratify = y_val_test)

In [ ]:
#https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader
#https://discuss.pytorch.org/t/apply-transform-on-numpy-arrays/19106/9
#https://wikidocs.net/157285
#https://stackoverflow.com/questions/46586616/pytorch-trying-to-apply-the-transform-to-a-numpy-array-fails-with-an-error
#https://076923.github.io/posts/Python-pytorch-11/
class FromNPDataset(torch.utils.data.Dataset):
  def __init__(self, x, y, transform = None):
    self.x = x
    self.y = y
    self.transform = transform
    
  def __getitem__(self, index):
    x = self.x[index]
    y = self.y[index]
    
    if self.transform:
      x = self.transform(x)
    else: 
      x = torch.FloatTensor(x)

    y = torch.Tensor(y)
    
    return x, y

  def __len__(self):
    return len(self.y)

In [ ]:
#from torchvision.transforms.transforms import ToPILImage

#Statistics Based on ImageNet Data for Normalisation (channel, width, height)
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

resize_dim = 224

data_transforms = {'train':transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize((resize_dim,resize_dim)), #Resizes all images into same dimension with the training data of ResNet
                                transforms.CenterCrop(resize_dim), #Performs Crop at Center and resizes it to 224 x 224
                                transforms.RandomRotation(15), # Rotates the images upto Max of 10 Degrees
                                # transforms.RandomHorizontalFlip(p=0.4), #Performs Horizantal Flip over images 
                                transforms.ToTensor(), # Coverts into Tensors
                                transforms.Normalize(mean = mean_nums, std=std_nums)]), # Normalizes
                    'val': transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize((resize_dim,resize_dim)),
                                transforms.CenterCrop(resize_dim), #Performs Crop at Center and resizes it to 224 x 224
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean_nums, std = std_nums)
                                ]),
                    'test': transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize((resize_dim,resize_dim)),
                                transforms.CenterCrop(resize_dim), #Performs Crop at Center and resizes it to 224 x 224
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean_nums, std = std_nums)
                                ])
                    }


In [ ]:
#Prepare torch dataset

#ds_train = FromNPDataset(x_train, y_train, transform = None)
#ds_val = FromNPDataset(x_val, y_val, transform = None)
#ds_test = FromNPDataset(x_test, y_test, transform = None)

ds_train = FromNPDataset(x_train, y_train, transform = data_transforms['train'])
ds_val = FromNPDataset(x_val, y_val, transform = data_transforms['val'])
ds_test = FromNPDataset(x_test, y_test, transform = data_transforms['test'])

**Defining the model**

In [ ]:
import torchvision.models as models
import torch.nn as nn

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [ ]:
output_dim = 2
intput_channel = 3
input_dim = 224*224*3

In [ ]:
#model = models.resnet50(pretrained=True)
#for param in model.parameters():
    #param.requires_grad = False

In [ ]:
def build_model(fc_layer_size, dropout):
  model = models.resnet50(pretrained=True)
  for param in model.parameters():
    param.requires_grad = False
  model.fc = nn.Sequential(nn.Linear(2048, fc_layer_size),
                           nn.ReLU(inplace=True),
                           nn.Dropout(dropout),
                           nn.Linear(fc_layer_size, output_dim)
                           )
  return model.to(device)
#model.to(device)

**Configuration of Training**

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.4 MB/s 
     |████████████████████████████████| 168 kB 58.4 MB/s 
     |████████████████████████████████| 182 kB 55.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 69.9 MB/s 
     |████████████████████████████████| 166 kB 56.6 MB/s 
     |████████████████████████████████| 162 kB 69.0 MB/s 
     |████████████████████████████████| 162 kB 67.5 MB/s 
     |████████████████████████████████| 158 kB 76.3 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 64.4 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 67.1 MB/s 
     |████████████████████████████████| 157 kB 66.0 MB/s 
     |████████████████████████████████| 157 kB 65.1 MB/s 
     |████████████████████████████

In [ ]:
import wandb
wandb.login()#(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=jtg_7FxLDVcm

# Define the Sweep

Fundamentally, a Sweep combines a strategy for trying out a bunch of hyperparameter values with the code that evalutes them.
Whether that strategy is as simple as trying every option
or as complex as [BOHB](https://arxiv.org/abs/1807.01774),
Weights & Biases Sweeps have you covered.
You just need to _define your strategy_
in the form of a [configuration](https://docs.wandb.com/sweeps/configuration).

When you're setting up a Sweep in a notebook like this,
that config object is a nested dictionary.
When you run a Sweep via the command line,
the config object is a
[YAML file](https://docs.wandb.com/sweeps/quickstart#2-sweep-config).

Let's walk through the definition of a Sweep config together.
We'll do it slowly, so we get a chance to explain each component.
In a typical Sweep pipeline,
this step would be done in a single assignment.

### 👈 Pick a `method`

The first thing we need to define is the `method`
for choosing new parameter values.

We provide the following search `methods`:
*   **`grid` Search** – Iterate over every combination of hyperparameter values.
Very effective, but can be computationally costly.
*   **`random` Search** – Select each new combination at random according to provided `distribution`s. Surprisingly effective!
*   **`bayes`ian Search** – Create a probabilistic model of metric score as a function of the hyperparameters, and choose parameters with high probability of improving the metric. Works well for small numbers of continuous parameters but scales poorly.

We'll stick with `random`.

In [ ]:
sweep_config = {
    'method':'random'
}
metric = {
    'name': 'loss',
    'goal':'minimize'
}
sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_layer_size' : {
        'values': [128, 256, 512]
    },
    'dropout': {
        'values' : [0.3, 0.4, 0.5]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
parameters_dict.update({
    'epochs': {
        'value': 10} ###
    })

In [ ]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
        }
        })

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 10},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [ ]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                                    lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                                     lr=learning_rate)
    return optimizer

In [ ]:
def train_epoch(network, loader, optimizer, criterion):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = criterion(network(data), target)
        cumu_loss += loss.item()#*data.size(0) #loss multiplied by the first dimension of inputs

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [ ]:
def evaluate(model, criterion, loader):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum of all loss values
            loss_ = criterion(output, target)
            loss += loss_.item()*data.size(0) #loss multiplied by the first dimension of inputs
            
            # the class with the maximal value is the predicted value
            pred = output.max(1, keepdim=True)[1]
            target_label = target.max(1, keepdim=True)[1]
            correct += (target_label==pred).sum().item()
            
    loss /= len(loader.dataset)
    accuracy = 100. * correct / len(loader.dataset)
    return loss, accuracy

In [ ]:
test_loader = torch.utils.data.DataLoader(
    dataset = ds_test,
    #batch_size = batch_size,
    shuffle = True
)

criterion = nn.CrossEntropyLoss()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="MedicalML-wk12-wandb-2022-Nov-19")

Create sweep with ID: lyi6030p
Sweep URL: https://wandb.ai/chandryou/MedicalML-wk12-wandb-2022-Nov-19/sweeps/lyi6030p


In [ ]:
import torch.nn.functional as F
def train(config = None):
    #initialize a new wandb run
    with wandb.init(config = config):
      config = wandb.config

      loader = torch.utils.data.DataLoader(dataset = ds_train,
                                           batch_size = config.batch_size,
                                           shuffle = True)
      model = build_model(config.fc_layer_size, config.dropout)
      optimizer = build_optimizer(model, config.optimizer, config.learning_rate)

      for epoch in range(config.epochs):
        avg_loss = train_epoch(model, loader, optimizer, criterion)
        wandb.log ({"loss":avg_loss, "epoch":epoch})

        val_loss, val_accuracy = evaluate(model, criterion, 
                                          torch.utils.data.DataLoader(dataset = ds_val,
                                                                      batch_size = config.batch_size,
                                                                      shuffle = True
                                                                      ))
        wandb.log ({"val_loss":val_loss, "val_accuracy":val_accuracy})

In [ ]:
wandb.agent(sweep_id, train, count=5) ###

wandb: Agent Starting Run: 94nsfkdf with config:
wandb: 	batch_size: 160
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.06241830935851335
wandb: 	optimizer: sgd
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chandryou. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

batch loss,█▆▆▅█▄▇▄▆▄▄▂▆▄▃▆▃▃▃▁▃▂▃▅▄▂▁▂▃▁▂▃▃▂▅▂▂▃▃▆
epoch,▁▂▃▃▄▅▆▆▇█
loss,▆█▄▄▃▂▁▁▁▁
val_accuracy,▁▅▆▇▇▇████
val_loss,█▅▃▃▃▃▁▁▁▁
batch loss,0.60898
epoch,9
loss,0.30964
val_accuracy,85.54846
val_loss,0.34212


wandb: Agent Starting Run: wolufkh4 with config:
wandb: 	batch_size: 160
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.020687966792536205
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▇▇▇█████
val_loss,█▇▃▄▃▁▁▁▁▁
batch loss,0.33107
epoch,9
loss,0.26782
val_accuracy,85.72258
val_loss,0.34461


wandb: Agent Starting Run: fbqdor78 with config:
wandb: 	batch_size: 96
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.07002178081164272
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [ ]:
wandb.finish()